In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy import stats
from IPython.display import HTML

np.random.seed(1)

# True parameters
mu_ctrl, mu_case, sigma = 0.0, 0.8, 1.0
ns = [2, 3, 4, 5, 6, 7, 8, 9, 10]
B = 2000

# Simulate once, and reuse subsets
ctrl_full = np.random.normal(mu_ctrl, sigma, size=15)
case_full = np.random.normal(mu_case, sigma, size=15)

fig, ax = plt.subplots(figsize=(6, 4))

def make_distributions(n, ctrl_full, case_full, rng):
    ctrl = ctrl_full[:n]
    case = case_full[:n]
    data = np.concatenate([ctrl, case])
    labels = np.array([0]*n + [1]*n)

    d_obs = case.mean() - ctrl.mean()

    pooled_var = ((n-1)*ctrl.var(ddof=1) + (n-1)*case.var(ddof=1)) / (2*n - 2)
    se = np.sqrt(pooled_var * (1/n + 1/n))

    perm_d = np.empty(1000)
    for b in range(1000):
        rng.shuffle(labels)
        perm_case = data[labels == 1]
        perm_ctrl = data[labels == 0]
        perm_d[b] = perm_case.mean() - perm_ctrl.mean()

    return d_obs, se, perm_d

def update(frame):
    ax.clear()
    n = ns[frame]
    rng = np.random.default_rng(123)  # fixed for reproducibility

    d_obs, se, perm_d = make_distributions(n, ctrl_full, case_full, rng)

    xmin = min(perm_d.min(), -4*se, d_obs - 4*se)
    xmax = max(perm_d.max(),  4*se, d_obs + 4*se)
    x = np.linspace(xmin, xmax, 400)

    param_pdf = stats.norm(0, se).pdf(x)
    ax.plot(x, param_pdf, lw=2, label="Parametric null (normal)")
    ax.hist(perm_d, bins=30, density=True, alpha=0.4, label="Permutation null")
    ax.axvline(d_obs, color="black", linestyle="--", label=f"Observed diff = {d_obs:.2f}")

    ax.set_title(f"Parametric vs permutation nulls\nn = {n} per group")
    ax.set_xlabel("Difference in means (case - ctrl)")
    ax.set_ylabel("Density")
    ax.legend(loc="upper left")
    ax.set_xlim(xmin, xmax)
    return []

anim = FuncAnimation(fig, update, frames=len(ns), interval=1500, blit=False)
anim.save("param_vs_perm.gif", writer=PillowWriter(fps=1))
HTML(anim.to_jshtml())
